# Variable Elimination in Student Bayes Model

The Student Network, based on https://uol.de/en/lcs/probabilistic-programming/webchurch-and-openbugs/example-6a-bayesian-network-student-model-with-evidence/ work and publication  

Also, we use pgmpy and the work of Ankur Ankan https://conference.scipy.org/proceedings/scipy2015/pdfs/ankur_ankan.pdf and https://scipy.in/2014/static/uploads/ankurankan/attachment/probabilistic_1.pdf

![alt text](sb.png)

### Importing libraries

In [1]:
from pgmpy.models.BayesianModel import BayesianModel
#from pgmpy.factors import TabularCPD
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

### Defining models

In [2]:
student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

grade_cpd = TabularCPD(variable='G', variable_card=3, values=[[0.3, 0.05, 0.9, 0.5], [0.4, 0.25, 0.08, 0.3], [0.3, 0.7, 0.02, 0.2]], evidence=['I', 'D'], evidence_card=[2, 2])
difficulty_cpd = TabularCPD(variable='D', variable_card=2, values=[[0.6, 0.4]])
intel_cpd = TabularCPD(variable='I', variable_card=2, values=[[0.7, 0.3]])
letter_cpd = TabularCPD(variable='L', variable_card=2, values=[[0.1, 0.4, 0.99], [0.9, 0.6, 0.01]], evidence=['G'], evidence_card=[3])
sat_cpd = TabularCPD(variable='S', variable_card=2, values=[[0.95, 0.2], [0.05, 0.8]], evidence=['I'], evidence_card=[2])


In [3]:
student_model.add_cpds(grade_cpd, difficulty_cpd, intel_cpd, letter_cpd, sat_cpd)
student_model.get_cpds()

[<TabularCPD representing P(G:3 | I:2, D:2) at 0x7fb8a51ca2b0>,
 <TabularCPD representing P(D:2) at 0x7fb8a51ca2e8>,
 <TabularCPD representing P(I:2) at 0x7fb8d445fb70>,
 <TabularCPD representing P(L:2 | G:3) at 0x7fb8a51ca240>,
 <TabularCPD representing P(S:2 | I:2) at 0x7fb8a51ca320>]

In [4]:
student_model.active_trail_nodes('D')

{'D': {'D', 'G', 'L'}}

In [5]:
student_model.local_independencies('G')

(G _|_ S | I, D)

In [6]:
student_model.get_independencies()

(D _|_ S, I)
(D _|_ I | S)
(D _|_ S | I)
(D _|_ L | G)
(D _|_ L | S, G)
(D _|_ S | I, L)
(D _|_ S, L | I, G)
(D _|_ L | S, I, G)
(D _|_ S | I, L, G)
(G _|_ S | I)
(G _|_ S | I, L)
(G _|_ S | I, D)
(G _|_ S | I, L, D)
(I _|_ D)
(I _|_ D | S)
(I _|_ L | G)
(I _|_ L | S, G)
(I _|_ L | G, D)
(I _|_ L | S, G, D)
(L _|_ S | I)
(L _|_ S, I, D | G)
(L _|_ I, D | S, G)
(L _|_ S, D | I, G)
(L _|_ S | I, D)
(L _|_ S, I | G, D)
(L _|_ D | S, I, G)
(L _|_ I | S, G, D)
(L _|_ S | I, G, D)
(S _|_ D)
(S _|_ L, G, D | I)
(S _|_ L | G)
(S _|_ G, D | I, L)
(S _|_ L, D | I, G)
(S _|_ L, G | I, D)
(S _|_ L | G, D)
(S _|_ D | I, L, G)
(S _|_ G | I, L, D)
(S _|_ L | I, G, D)

### Queries on Variable Elimination

In [7]:
ve_model = VariableElimination(student_model)
prob_SI = ve_model.query(variables=['S','I'])

/home/jenazads/anaconda3/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
/home/jenazads/anaconda3/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


In [8]:
print(prob_SI['I'])

+-----+----------+
| I   |   phi(I) |
+=====+==========+
| I_0 |   0.7000 |
+-----+----------+
| I_1 |   0.3000 |
+-----+----------+


In [9]:
#prob_G = ve_model.query(variables=['G'], evidence=[('I', 1), ('D', 0)])
prob_G = ve_model.query(variables=['G'])

/home/jenazads/anaconda3/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


In [10]:
print(prob_G['G'])

+-----+----------+
| G   |   phi(G) |
+=====+==========+
| G_0 |   0.3620 |
+-----+----------+
| G_1 |   0.2884 |
+-----+----------+
| G_2 |   0.3496 |
+-----+----------+


In [11]:
ve_model.map_query(variables=['G'])

/home/jenazads/anaconda3/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


{'G': 0}

In [12]:
#ve_model.map_query(variables=['G'], evidence=[('I', 1, ('D', 0)])